<a href="https://colab.research.google.com/github/twillixa/HEC/blob/main/PPS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

!pip install pulp numpy pandas matplotlib
import numpy as np
import pandas as pd



In [ ]:
def simulate_plan(
    demand,
    production_per_worker_regular,
    starting_inventory,
    desired_ending_inventory,
    initial_workers,
    wage_per_worker,
    hire_cost,
    layoff_cost,
    inventory_cost_per_unit,
    overtime_months,
    overtime_percentage,
    overtime_wage_multiplier,
    strategy,
):
    months = len(demand)
    overtime_months = set(overtime_months or [])
    demand = np.array(demand, dtype=float)

    # workforce arrays
    workers = np.zeros(months)
    hired = np.zeros(months)
    laid_off = np.zeros(months)
    inventory = np.zeros(months + 1)
    inventory[0] = starting_inventory

    # helper: overtime factor by month
    ot_flag = np.array([1.0 if t in overtime_months else 0.0 for t in range(months)])
    prod_per_worker = production_per_worker_regular * (1 + ot_flag * overtime_percentage)

    # choose workforce rule
    if strategy == "chase":
        workers_needed = np.ceil(demand / production_per_worker_regular)
    else:
        total_required = demand.sum() + desired_ending_inventory - starting_inventory
        total_capacity_per_worker = prod_per_worker.sum()
        w_const = int(np.ceil(total_required / total_capacity_per_worker))
        workers_needed = np.full(months, w_const)

    # simulate month-by-month
    for t in range(months):
        workers[t] = workers_needed[t]
        if t == 0:
            hired[t] = max(0, workers[t] - initial_workers)
            laid_off[t] = max(0, initial_workers - workers[t])
        else:
            hired[t] = max(0, workers[t] - workers[t-1])
            laid_off[t] = max(0, workers[t-1] - workers[t])
        produced = workers[t] * prod_per_worker[t]
        inventory[t+1] = inventory[t] + produced - demand[t]

    # costs
    wage_cost = workers * wage_per_worker
    hiring_cost = hired * hire_cost
    layoff_costs = laid_off * layoff_cost
    inv_cost = inventory[1:] * inventory_cost_per_unit
    overtime_cost = workers * wage_per_worker * (ot_flag * overtime_percentage) * overtime_wage_multiplier

    total_monthly = wage_cost + hiring_cost + layoff_costs + inv_cost + overtime_cost

    df = pd.DataFrame({
        "Month": np.arange(1, months+1),
        "Workers": workers,
        "Hired": hired,
        "Laid off": laid_off,
        "Produced": workers * prod_per_worker,
        "Demand": demand,
        "End Inventory": inventory[1:],
        "Wage Cost": wage_cost,
        "Overtime Cost": overtime_cost,
        "Hiring Cost": hiring_cost,
        "Layoff Cost": layoff_costs,
        "Inventory Cost": inv_cost,
        "Total Cost": total_monthly,
    })

    summary = {
        "Total Wage": float(wage_cost.sum()),
        "Total Overtime": float(overtime_cost.sum()),
        "Total Hiring": float(hiring_cost.sum()),
        "Total Layoff": float(layoff_costs.sum()),
        "Total Inventory": float(inv_cost.sum()),
        "Grand Total": float(total_monthly.sum()),
    }

    return df, summary

In [ ]:
df_level, s_level = simulate_plan(
    demand                        = [300, 300, 350, 400, 450, 500, 650, 600, 475, 475, 450, 450],
    production_per_worker_regular = 10,
    starting_inventory            = 50,
    desired_ending_inventory      = 50,
    initial_workers               = 40,
    wage_per_worker               = 2,
    hire_cost                     = 1,
    layoff_cost                   = 2,
    inventory_cost_per_unit       = 0.06,
    overtime_months               = [5, 6, 7],
    overtime_percentage           = 0.0,
    overtime_wage_multiplier      = 1.5,
    strategy                      = "level",
)
display(df_level)
s_level

,Month,Workers,Hired,Laid off,Produced,Demand,End Inventory,Wage Cost,Overtime Cost,Hiring Cost,Layoff Cost,Inventory Cost,Total Cost
0,1,45.0,5.0,0.0,450.0,300.0,200.0,90.0,0.0,5.0,0.0,12.0,107.0
1,2,45.0,0.0,0.0,450.0,300.0,350.0,90.0,0.0,0.0,0.0,21.0,111.0
2,3,45.0,0.0,0.0,450.0,350.0,450.0,90.0,0.0,0.0,0.0,27.0,117.0
3,4,45.0,0.0,0.0,450.0,400.0,500.0,90.0,0.0,0.0,0.0,30.0,120.0
4,5,45.0,0.0,0.0,450.0,450.0,500.0,90.0,0.0,0.0,0.0,30.0,120.0
5,6,45.0,0.0,0.0,450.0,500.0,450.0,90.0,0.0,0.0,0.0,27.0,117.0
6,7,45.0,0.0,0.0,450.0,650.0,250.0,90.0,0.0,0.0,0.0,15.0,105.0
7,8,45.0,0.0,0.0,450.0,600.0,100.0,90.0,0.0,0.0,0.0,6.0,96.0
8,9,45.0,0.0,0.0,450.0,475.0,75.0,90.0,0.0,0.0,0.0,4.5,94.5
9,10,45.0,0.0,0.0,450.0,475.0,50.0,90.0,0.0,0.0,0.0,3.0,93.0


{'Total Wage': 1080.0,
 'Total Overtime': 0.0,
 'Total Hiring': 5.0,
 'Total Layoff': 0.0,
 'Total Inventory': 181.5,
 'Grand Total': 1266.5}